## Improve Model Performance & Submission

This notebook was created for the purpose of organizing the contents of [the book "Machine Learning Deep Learning Problem Solving Strategy"](https://goldenrabbit.co.kr/product/must-have-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%C2%B7%EB%94%A5%EB%9F%AC%EB%8B%9D-%EB%AC%B8%EC%A0%9C%ED%95%B4%EA%B2%B0-%EC%A0%84%EB%9E%B5/).

이 노트북은 [책 "머신러닝 딥러닝 문제해결 전략"](https://goldenrabbit.co.kr/product/must-have-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%C2%B7%EB%94%A5%EB%9F%AC%EB%8B%9D-%EB%AC%B8%EC%A0%9C%ED%95%B4%EA%B2%B0-%EC%A0%84%EB%9E%B5/) 내용을 정리하는 목적으로 만들어졌습니다.

.

베이스라인 모델을 만들어 제출까지 해봤습니다. 이번에는 사이킷런이 제공하는 모델 중 세가지(ridge, lasso, random forest 회귀)를 더 다뤄보며 가장 우수한 모델이 무엇인지 알아보겠습니다.

이 노트북은 연습용이다보니 [#3. 베이스라인 모델](https://www.kaggle.com/code/scottxchoo/3-baseline-model)의 3번인 Create an evaluation index calculation function (평가지표 계산 함수 작성)까지는 그대로 진행할 것입니다. 그러니 "#3. 베이스라인 모델 노트북"을 복사한 뒤, 3번까지 진행해주세요.

### Load the data (데이터 불러오기)

In [ ]:
import pandas as pd

data_path = '/kaggle/input/bike-sharing-demand/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

### Feature Engineering (피처 엔지니어링)

In [ ]:
# Extract only data where weather is not 4 from training data (훈련 데이터에서 weather가 4가 아닌 데이터만 추출)
train = train[train['weather'] != 4]

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data

In [ ]:
from datetime import datetime

# Create a date feature (날짜 피처 생성)
all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])

# Create a year feature (연도 피처 생성)
all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])

# Create a month feature (월 피처 생성)
all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1])

# Create a hour feature (시 피처 생성)
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])

# Create a weekday feature (요일 피처 생성)
all_data['weekday'] = all_data['date'].apply(lambda dateString :
datetime.strptime(dateString, "%Y-%m-%d").weekday())

In [ ]:
drop_features = ['casual', 'registered', 'datetime', 'date', 'windspeed', 'month']

all_data = all_data.drop(drop_features, axis = 1)

In [ ]:
# 1. Divide the training and test data (훈련 데이터와 테스트 데이터 나누기)
X_train = all_data[~pd.isnull(all_data['count'])]
X_test = all_data[pd.isnull(all_data['count'])]

# 2. Remove the target value 'count' (타깃값 'count' 제거)
X_train = X_train.drop(['count'], axis = 1)
X_test = X_test.drop(['count'], axis = 1)

# 3. The target value (타깃값)
y = train['count']

In [ ]:
X_train.head()

### Create an evaluation index calculation function (평가지표 계산 함수 작성)

In [ ]:
import numpy as np

def rmsle(y_true, y_pred, convertExp = True):
    # 1. Exponential Transformation (지수변환)
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
    
    # 2. After log transformation, convert missing values to zero (로그변환 후 결측값을 0으로 변환)
    log_true = np.nan_to_num(np.log(y_true + 1))
    log_pred = np.nan_to_num(np.log(y_pred + 1))
    
    # 3. Calculate RMSLE (RMSLE 계산)
    output = np.sqrt(np.mean((log_true - log_pred) ** 2))
    return output

## [1] Ridge Regression Model (릿지 회귀 모델)

릿지 회귀 모델은 L2 규제를 적용한 선형 회귀 모델입니다. 규제란 모델이 훈련 데이터에 과대적합(overfitting)되지 않도록 해주는 방법입니다. 훈련 데이터에 과대적합되면 모델이 훈련 데이터에만 너무 잘 들어맞고, 테스트 데이터로는 제대로 예측하지 못합니다. 따라서 모델이 과대적합되지 않게 훈련하는 게 중요합니다. 규제는 이럴 때 사용하는 방법입니다.

릿지 회귀 모델은 성능이 좋은 편은 아닙니다. 단순 선형 회귀 모델보다 과대적합이 적은 모델 정도로 생각하면 됩니다.

### Improve Model Performance Process
1. Load the data (데이터 불러오기)
2. Feature Engineering (피처 엔지니어링)
3. Create an evaluation index calculation function (평가지표 계산 함수 작성)
4. Optimize hyperparameters (하이퍼 파라미터 최적화)
5. Validate model performance (모델 성능 검증)
6. Submission (제출)

위에서 얘기한 것처럼 3번까지는 베이스라인 모델과 같기에 4번부터 진행하겠습니다.

### Optimize hyperparameters (하이퍼 파라미터 최적화)

이번 단계에서 grid search 기법을 사용할 것입니다. grid search는 하이퍼파라미터를 격자처럼 촘촘하게 순회하며 최적의 하이퍼파라미터 값을 찾는 기법입니다. 각 하이퍼파라미터를 적용한 모델마다 교차 검증(cross-validation)하며 성능을 측정하여 최종적으로 성능이 가장 좋았을 때의 하이퍼파라미터 값을 찾아줍니다. 교차 검증 평가점수는 보통 에러 값이기 때문에 낮을수록 좋습니다.

그리드서치를 이용하지 않으면 수작업으로 하나하나 수행한 뒤, 최적 하이퍼파라미터를 찾아야 하니 무척 번거롭습니다. 특히 하이퍼파라미터의 개수가 하나 늘어날 때마다 번거로움은 기하급수적으로 커질 것입니다. 그리드서치는 이 일을 자동으로 해줍니다. 테스트하려는 하이퍼파라미터와 값의 범위만 전달하면 알아서 모든 가능한 조합을 순회하며 교차 검증합니다.

이제 그리드서치로 최적의 릿지 회귀 모델을 찾아내는 코드를 살펴봅시다.

#### 모델 생성

가장 먼저 릿지 모델을 생성합니다.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

ridge_model = Ridge()

특별한 것 없이 사이킷런의 기본 릿지 모델을 생성했습니다.

#### 그리드서치 객체 생성

이어서 그리드서치 객체를 생성합니다. 앞서 그리드서치는 '하이퍼파라미터의 값'을 바꿔가며 '모델'의 성능을 교차 검증으로 '평가'해 최적의 하이퍼파라미터 값을 찾아준다고 했습니다. 이 말은 그리드서치 객체가 다음의 세 가지를 알고 있어야 한다는 뜻입니다.

1. 비교 검증해볼 하이퍼파라미터 값 목록
2. 대상 모델
3. 교차 검증용 평가 수단(평가 함수)

대상 모델은 앞서 만들었으니, 하이퍼파라미터 값 목록과 평가 함수만 더 준비하면 됩니다.

릿지 모델은 규제를 적용한 회귀 모델이라고 했습니다. 릿지 모델에서 중요한 하이퍼파라미터는 alpha로, 값이 클수록 규제 강도가 세집니다. 적절한 규제를 적용한다면, 즉 alpha를 적당한 크기로 하면 과대적합 문제를 개선할 수 있습니다.

In [ ]:
# 하이퍼파라미터 값 목록
ridge_params = {'max_iter' : [3000], 'alpha' : [0.1, 1, 2, 3, 4, 10, 30, 100, 200, 300, 400, 800, 900, 1000]}

# 교차 검증용 평가 함수(RMSLE 점수 계산)
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better = False)

# 그리드서치(with 릿지) 객체 생성
gridsearch_ridge_model = GridSearchCV(estimator = ridge_model,   # 릿지 모델
                                      param_grid = ridge_params, # 값 목록
                                      scoring = rmsle_scorer,    # 평가지표
                                      cv = 5)                    # 교차 검증 분할 수

그리드서치 객체를 생성하는 GridSearchCV() 함수의 주요 파라미터는 다음과 같습니다.
- estimator : 분류 및 회귀 모델
- param_grid : 딕셔너리 형태로 모델의 하이퍼파라미터명과 여러 하이퍼파라미터 값을 지정
- scoring : 평가지표.
- cv : 교차 검증 분할 개수(기본값은 5)

하이퍼파라미터 값 목록에서 max_iter는 3000으로 고정했고, alpha는 0.1에서 1000까지 다양합니다. 그리드서치 객체는 param_grid로 전달된 모든 하이퍼파라미터를 대입해 교차 검증으로 모델 성능 점수를 계산하여 어떤 값일 때 점수가 가장 좋은지 찾아줍니다.

교차 검증 시에는 해당 경진대회의 평가지표를 그대로 사용해야 합니다. 평가 방식이 다르면 애써 찾은 최적의 하이퍼파라미터가 대회 성적과는 무관한 쪽으로 우수한 예측 결과를 내어줄 것입니다.

#### 그리드서치 수행

방금 만든 그리드서치 객체를 이용하여 그리드서치를 수행합니다.

In [ ]:
log_y = np.log(y) # 타깃값 로그변환
gridsearch_ridge_model.fit(X_train, log_y) # 훈련(그리드서치)

코드가 일관되도록 그리드서치 객체도 모델 객체와 똑같이 fit() 메서드를 제공합니다. fit()을 실행하면 객체 생성 시 param_gid에 전달된 값들을 순회하면서 교차 검증으로 평가지표 점수를 계산합니다. 이때 가장 좋은 성능을 보인 값을 best_params_ 속성에 저장하며, 이 최적 값으로 훈련한 모델(최적 예측기)을 best_estimator_ 속성에 저장합니다.

그렇다면 최적 하이퍼파라미터로는 어떤 값이 선정되었는지 살펴봅시다.

In [ ]:
print('최적 하이퍼파라미터 :', gridsearch_ridge_model.best_params_)

출력 결과를 보면 alpha가 0.1이고 max_iter가 3000일 때 가장 좋은 성능을 낸다는 사실을 알 수 있습니다.

### Validate model performance (모델 성능 검증)

그리드서치를 완료하고 나면 그리드서치 객체의 best_estimator_ 속성에 최적 예측기가 저장되어 있습니다. 따라서 예측은 그리드서치 객체의 best_estimator_ 속성에 저장된 모델로 수행하면 됩니다.

In [ ]:
# 예측
preds = gridsearch_ridge_model.best_estimator_.predict(X_train)

# 평가
print(f'릿지 회귀 RMSLE 값 : {rmsle(log_y, preds, True): .4f}')

참 값(log_y)과 예측값(preds) 사이의 RMSLE는 1.02로, 선형 회귀 모델의 결과와 다르지 않음을 알 수 있습니다.

## [2] Lasso Regression Model (라쏘 회귀 모델)

라쏘 회귀 모델은 L1 규제를 적용한 선형 회귀 모델입니다. 나중에 여러분이 직접 만드는 모델은 적어도 릿지나 라쏘보다는 성능이 좋아야 할 것입니다.

### Optimize hyperparameters (하이퍼 파라미터 최적화)

사용한 모델과 파라미터만 다를 뿐 릿지 회귀 때와 똑같은 흐름입니다(rmsle_scorer 함수는 릿지 회귀 때 정의한 것을 재활용했습니다). 릿지 회귀와 마찬가지로 alpha는 규제 강도를 조정하는 파라미터입니다. 

In [ ]:
from sklearn.linear_model import Lasso

# 모델 생성
lasso_model = Lasso()
# 하이퍼파라미터 값 목록
lasso_alpha = 1 / np.array([0.1, 1, 2, 3, 4, 10, 30, 100, 200, 300, 400, 800, 900, 1000])
lasso_params = {'max_iter': [3000], 'alpha': lasso_alpha}
# 그리드서치(with lasso) 객체 생성
gridsearch_lasso_model = GridSearchCV(estimator = lasso_model,
                                      param_grid = lasso_params,
                                      scoring = rmsle_scorer,
                                      cv = 5)

# 그리드서치 수행
log_y = np.log(y)
gridsearch_lasso_model.fit(X_train, log_y)

print('최적 하이퍼파라미터 :', gridsearch_lasso_model.best_params_)

### Validate model performance (모델 성능 검증)

그리드서치로 찾은 최적 예측기로 예측하여 RMSLE 값을 확인해보겠습니다.

In [ ]:
# 예측
preds = gridsearch_lasso_model.best_estimator_.predict(X_train)

# 평가
print(f'라쏘 회귀 RMSLE 값 : {rmsle(log_y, preds, True): .4f}')

여전히 개선되지 않았습니다.

## [3] 랜덤 포레스트 회귀 모델

마지막으로 랜덤 포레스트 회귀 모델을 사용해보겠습니다. 랜덤 포레스트 회귀는 간단히 생각하면 훈련 데이터를 랜덤하게 샘플링한 모델 n개를 각각 훈련하여 결과를 평균하는 방법입니다.

### Optimize hyperparameters (하이퍼 파라미터 최적화)

랜덤 포레스트 회귀 모델로 그리드서치를 수행하고 최적 하이퍼파라미터 값까지 출력해보겠습니다.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 모델 생성
randomforest_model = RandomForestRegressor()
# 그리드서치 객체 생성
rf_params = {'random_state' : [42], 'n_estimators' : [100, 120, 140]}
gridsearch_random_forest_model = GridSearchCV(estimator = randomforest_model,
                                              param_grid = rf_params,
                                              scoring = rmsle_scorer,
                                              cv = 5)

# 그리드서치 수행
log_y = np.log(y)
gridsearch_random_forest_model.fit(X_train, log_y)
print('최적 하이퍼파라미터 :', gridsearch_random_forest_model.best_params_)

random_state는 랜덤 시드값으로, 값을 명시하면 코드를 다시 실행해도 같은 결과를 얻을 수 있습니다.

n_estimators는 랜덤 포레스트를 구성하는 결정 트리 개수를 의미합니다.

### Validate model performance (모델 성능 검증)

최적 예측기의 성능을 확인해보죠.

In [ ]:
# 예측
preds = gridsearch_random_forest_model.best_estimator_.predict(X_train)

# 평가
print(f'랜덤 포레스트 회귀 RMSLE 값 : {rmsle(log_y, preds, True): .4f}')

랜덤 포레스트 회귀 모델을 사용하니 RMSLE 값이 큰 폭으로 개선되었네요. 선형 회귀, 릿지 회귀, 라쏘 회귀 모델의 RMSLE 값은 모두 1.02였습니다. 반면 랜덤 포레스트 회귀 모델은 0.11입니다(값이 작을수록 좋습니다). 네 모델 중 성능이 가장 좋은 모델은 랜덤 포레스트입니다.

### Submission (제출)

성능 측정을 훈련 데이터로 했기 때문에 테스트 데이터에서도 성능이 좋다고 보장할 수는 없습니다. 훈련 데이터와 테스트 데이터의 분포가 비슷하면 과대적합 문제가 상대적으로 적기 때문에 훈련 데이터에서 성능이 좋다면 테스트 데이터에서도 좋은 가능성이 큽니다.

검증 결과 랜덤 포레스트 회귀 모델의 성능이 가장 좋았습니다.

In [ ]:
randomforest_preds = gridsearch_random_forest_model.best_estimator_.predict(X_test)

submission['count'] = np.exp(randomforest_preds) # 지수 변환
submission.to_csv('submission.csv', index = False)

The End.